In [43]:
import notebook_common
import jaconv
from importlib import reload

reload(notebook_common)
from notebook_common import (
    # ------------------function------------------
    # read_excel,
    to_csv_sig,
    # ------------------2400 word------------------
    # r_word_2400_xlsx,
    # w_word_2400_csv,
    # w_word_2400_with_guide_csv,
    # w_word_2400_without_guide_csv,
    # ------------------JLPT word------------------
    r_jlpt_csv,
    r_jlpt_v2_csv,
    w_jlpt_csv,
    w_jlpt_word_csv,
    w_jlpt_multiidx_agg_csv,
    # ------------------书面语相关------------------
    # 解析书面语正则
    # pattern_table_written,
    # pattern_td_written,
    # r_written_freq_nlt_with_enter_txt,
    # w_written_freq_nlt_with_enter_csv,
    # w_written_freq_nlt_sumed_csv,
    # ------------------口语相关------------------
    # 解析口头语正则
    # pattern_table_spoken,
    # pattern_td_spoken,
    # r_spoken_freq_nlb_with_enter_txt,
    # w_spoken_freq_nlb_with_enter_csv,
    # w_spoken_freq_nlb_with_enter_filtered_csv,
    # w_spoken_freq_nlb_with_enter_filtered_hrt_csv,
    # w_spoken_freq_nlb_sumed_csv,
    # ------------------合并书面语和口语词频------------------
    # w_merged_freq_sumed_csv,
    w_raw_merged_freq_sumed_csv,
    # ------------------公共python库------------------
    pd,
    np,
    p,
    re,
    l,
    fetch_random_from_set,
)

l("Import common module")

----------------------------------------Import common module----------------------------------------


In [44]:
# 读取原始 jlpt文件
df_jlpt = pd.read_csv(
    r_jlpt_v2_csv,
    encoding="utf-8-sig",
    on_bad_lines="skip",
    engine="python",
    keep_default_na=False,
    # index_col=["word", "reading"]
)

# 把JLPT中的level替换成1到6
df_jlpt['level']=df_jlpt['level'].str[:1]
df_jlpt['level']=df_jlpt['level'].astype('float32')

# 把JLPT中的片假名转为平假名
df_jlpt['hira']=df_jlpt['hira'].apply(jaconv.kata2hira)


df_abc = df_jlpt.copy()


# 读原始书面语和口语词频合并
df_merged_raw = pd.read_csv(
    w_raw_merged_freq_sumed_csv,
    encoding="utf-8-sig",
    on_bad_lines="skip",
    engine="python",
    keep_default_na=False,
    # index_col=["headword", "reading"]
)
l("read ok")

----------------------------------------read ok----------------------------------------


In [45]:
l('jlpt')
df_jlpt

----------------------------------------jlpt----------------------------------------


,number,word,hira,level,multi_prop,prop1,sub_prop1,prop2,sub_prop2,prop3,sub_prop3,word_build,origin
0,4,あ,あ,2.0,,名词,,,,,,名词-普通名词-一般,日语固有词
1,3,亜,あ,6.0,,缀词(前),,,,,,名词-普通名词-一般,来自中文
2,5,ああ,ああ,2.0,,感叹词,,,,,,感叹词-一般,日语固有词
3,6,ああ,ああ,3.0,,感叹词,,,,,,副词,日语固有词
4,7,アーケード,あーけーど,4.0,,名词,,,,,,名词-普通名词-一般,来自西方
...,...,...,...,...,...,...,...,...,...,...,...,...,...
17903,19336,ワンパターン,わんぱたーん,5.0,,名词,,,,,,名词-普通名词-一般 ＋ 名词-普通名词-一般,来自西方
17904,20121,ワンピース,わんぴーす,2.0,,名词,,,,,,名词-普通名词-一般,来自西方
17905,19337,ワンマン,わんまん,5.0,,名词,,,,,,形状詞-一般,来自西方
17906,19338,腕力,わんりょく,5.0,,名词,,,,,,名词-普通名词-一般,来自中文


In [46]:
l("df_merged_raw")
df_merged_raw

----------------------------------------df_merged_raw----------------------------------------


,headword,hira,spoken_freq,roman,written_freq
0,の,の,57221425.0,no,686025.0
1,に,に,35286930.0,,
2,を,を,33707106.0,,
3,は,は,31279365.0,,
4,が,が,30705135.0,,
...,...,...,...,...,...
104314,8ミリビデオカメラ,はちみりびでおかめら,,hachimiribideokamera,1.0
104315,鼻緒する,はなおする,,hanaosuru,1.0
104316,DJする,でぃーじぇいする,,dhijeisuru,1.0
104317,CTB,しーてぃーびー,,shithibi,1.0


In [47]:
# jlpt中word 找出jlpt词汇里有多少个在词频表里

# 1.jlpt本身的数据
jlpt_shape = df_jlpt.shape
p(f"jlpt shape:{jlpt_shape}")
word_jlpt_set = set(df_jlpt["word"])
p(f"word size in jlpt:{len(word_jlpt_set)}")
hira_jlpt_set = set(df_jlpt["hira"])
p(f"hira size in jlpt:{len(hira_jlpt_set)}")
index_jlpt_set = set(df_jlpt.index)
p(f"index size in jlpt:{len(index_jlpt_set)}")

l()
# 2.合并书面语和口语原始数据
def_merged_raw_shape = df_merged_raw.shape
p(f"def_merged_raw_shape:{def_merged_raw_shape}")

headword_freq_merge_set = set(df_merged_raw["headword"])
p(f"raw merged headword size:{len(headword_freq_merge_set)}")
reading_freq_merge_set = set(df_merged_raw["hira"])
p(f"raw merged hira size:{len(reading_freq_merge_set)}")

jlpt shape:(17908, 13)
word size in jlpt:17207
hira size in jlpt:15592
index size in jlpt:17908
--------------------------------------------------------------------------------
def_merged_raw_shape:(104319, 5)
raw merged headword size:102381
raw merged hira size:80034


In [48]:
# 1. jlpt的word和词频表的headword的交集
word_intersection = word_jlpt_set & headword_freq_merge_set
len_word_jlpt_set = len(word_jlpt_set)
len_word_found = len(word_intersection)
per_word_found = len_word_found / len_word_jlpt_set
len_word_unfound = len_word_jlpt_set - len_word_found
per_word_unfound = len_word_unfound / len_word_jlpt_set

p(f"word size in jlpt:{len_word_jlpt_set}")
p(f"word_intersection:{len_word_found}({per_word_found:.1%})")
p(f"unfound word in jlpt:{len_word_unfound}({per_word_unfound:.1%})")
l()
# 2. jlpt的reading和词频表的hira的交集
hira_intersection = hira_jlpt_set & reading_freq_merge_set
len_hira_jlpt_set = len(hira_jlpt_set)
len_hira_found = len(hira_intersection)
per_hira_found = len_hira_found / len_hira_jlpt_set
len_hira_unfound = len_hira_jlpt_set - len_hira_found
per_hira_unfound = len_hira_unfound / len_hira_jlpt_set
p(f"hira size in jlpt:{len_hira_jlpt_set}")
p(f"hira_intersection:{len_hira_found}({per_hira_found:.1%})")
p(f"unfound hira in jlpt:{len_hira_unfound}({per_hira_unfound:.1%})")
l()
# 3. 计算jlpt中word和hira同时匹配词频
p(f"jlpt shape:{df_jlpt.shape}")
p(f"df_merged_raw shape:{df_merged_raw.shape}")


df_jlpt_with_multiidx = df_jlpt.set_index(["word", "hira"])
df_merged_raw_with_multiidx = df_merged_raw.set_index(["headword", "hira"])

idx_jlbt_set = set(df_jlpt_with_multiidx.index)
idx_df_merged_raw_set = set(df_merged_raw_with_multiidx.index)

l("df_jlpt_with_multiidx.index")
p(f"len df_jlpt_with_multiidx.index:{len(idx_jlbt_set)}")
fetch_random_from_set(idx_jlbt_set)
l("df_merged_raw_with_multiidx.index")
p(f"len idx_df_merged_raw_set.index:{len(idx_df_merged_raw_set)}")
fetch_random_from_set(idx_df_merged_raw_set)
l()
# jlpt的复合索引(word-reading)和词频表的复合索引的交集
index_intersection = idx_jlbt_set & idx_df_merged_raw_set
len_index_jlpt_set = len(idx_jlbt_set)
len_index_found = len(index_intersection)
per_index_found = len_index_found / len_index_jlpt_set
len_index_unfound = len_index_jlpt_set - len_index_found
per_index_unfound = len_index_unfound / len_index_jlpt_set
p(f"index size in jlpt:{len_index_jlpt_set}")
p(f"index_intersection:{len_index_found}({per_index_found:.1%})")
p(f"unfound index in jlpt:{len_index_unfound}({per_index_unfound:.1%})")
l()
df_jlpt_with_freq = df_jlpt.join(
    df_merged_raw, how="left", lsuffix="_jlpt", rsuffix="_freqdict"
)
to_csv_sig(df_jlpt_with_freq, "d:/tmp/h.csv")
p("df_jlpt_with_freq--ok")

word size in jlpt:17207
word_intersection:14753(85.7%)
unfound word in jlpt:2454(14.3%)
--------------------------------------------------------------------------------
hira size in jlpt:15592
hira_intersection:13896(89.1%)
unfound hira in jlpt:1696(10.9%)
--------------------------------------------------------------------------------
jlpt shape:(17908, 13)
df_merged_raw shape:(104319, 5)
----------------------------------------df_jlpt_with_multiidx.index----------------------------------------
len df_jlpt_with_multiidx.index:17496
[('柱', 'はしら'), ('救助', 'きゅうじょ'), ('セルフ', 'せるふ'), ('目印', 'めじるし'), ('義理', 'ぎり')]
----------------------------------------df_merged_raw_with_multiidx.index----------------------------------------
len idx_df_merged_raw_set.index:104319
[('セクショナリズム', 'せくしょなりずむ'), ('続ける-非自立', 'つづける'), ('帝位', 'ていい'), ('奥地', 'おくち'), ('うん', 'うん')]
--------------------------------------------------------------------------------
index size in jlpt:17496
index_intersection:14623(83.6%)


In [49]:
df_jlpt_with_multiidx["cnt_multiindex"] = df_jlpt_with_multiidx.groupby(
    df_jlpt_with_multiidx.index
).transform("size")
to_csv_sig(df_jlpt_with_multiidx, "d:/tmp/e.csv", True)
l("df_jlpt_with_multiidx export to d:/tmp/e.csv", 20)
df_jlpt_with_multiidx



--------------------df_jlpt_with_multiidx export to d:/tmp/e.csv--------------------


number  level multi_prop  prop1 sub_prop1 prop2 sub_prop2  \
word   hira                                                                
あ      あ            4    2.0                名词                             
亜      あ            3    6.0             缀词(前)                             
ああ     ああ           5    2.0               感叹词                             
       ああ           6    3.0               感叹词                             
アーケード  あーけーど        7    4.0                名词                             
...               ...    ...        ...    ...       ...   ...       ...   
ワンパターン わんぱたーん   19336    5.0                名词                             
ワンピース  わんぴーす    20121    2.0                名词                             
ワンマン   わんまん     19337    5.0                名词                             
腕力     わんりょく    19338    5.0                名词                             
わんわん   わんわん     19339    4.0                名词                             

              prop3 sub_prop3               word_build origin  cnt_multiindex  
word   hira                                                                    
あ      あ                                    名词-普通名词-一般  日语固有词               1  
亜      あ                                    名词-普通名词-一般   来自中文               1  
ああ     ああ                                       感叹词-一般  日语固有词               2  
       ああ                                           副词  日语固有词               2  
アーケード  あーけーど                                名词-普通名词-一般   来自西方               1  
...             ...       ...                      ...    ...             ...  
ワンパターン わんぱたーん                  名词-普通名词-一般 ＋ 名词-普通名词-一般   来自西方               1  
ワンピース  わんぴーす                                名词-普通名词-一般   来自西方               1  
ワンマン   わんまん                                     形状詞-一般   来自西方               1  
腕力     わんりょく                                名词-普通名词-一般   来自中文               1  
わんわん   わんわん                                         副词  日语固有词               1  

[17908 rows x 12 columns]

In [50]:
# 取出jlpt中索引不在词频表中的那些数据(复合索引)
df_2914 = df_jlpt_with_multiidx[
    ~df_jlpt_with_multiidx.index.isin(df_merged_raw_with_multiidx.index)
]
to_csv_sig(df_2914, "d:/tmp/d.csv", True)
l("df_2914 export to d:/tmp/d.csv", 20)
df_2914

--------------------df_2914 export to d:/tmp/d.csv--------------------


,,number,level,multi_prop,prop1,sub_prop1,prop2,sub_prop2,prop3,sub_prop3,word_build,origin,cnt_multiindex
word,hira,,,,,,,,,,,,
亜,あ,3,6.0,,缀词(前),,,,,,名词-普通名词-一般,来自中文,1
アーチスト,あーちすと,9,3.0,,名词,,,,,,名词-普通名词-一般,来自西方,1
愛煙家,あいえんか,17,6.0,,名词,,,,,,名词-普通名词-一般 ＋ 缀词(后)-名词的-一般,来自中文,1
ＩＱ,あいきゅー,20,5.0,,名词,,,,,,名词-普通名词-一般,来自西方,1
アイスコーヒー,あいすこーひー,40,1.0,,名词,,,,,,名词-普通名词-一般 ＋ 名词-普通名词-一般,来自西方,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
割に,わりに,19305,4.0,,副词,,,,,,名词-普通名词-可变副词 ＋ 助词-格助词,日语固有词,1
割りばし,わりばし,19306,3.0,,名词,,,,,,名词-普通名词-一般,日语固有词,1
割引,わりびき,20105,5.0,〇,名词,,动词,动3,,,名词-普通名词-可变动3,日语固有词,1


In [51]:
# 取出jlpt中索引不在词频表中的那些数据(只有拼写索引)
df_jlpt_with_word_idx = df_jlpt.set_index("word")
df_merged_raw_with_word_idx = df_merged_raw.set_index("headword")
df_2480 = df_jlpt_with_word_idx[
    ~df_jlpt_with_word_idx.index.isin(df_merged_raw_with_word_idx.index)
]
to_csv_sig(df_2480, "d:/tmp/e.csv", True)
l("df_2480 export to d:/tmp/e.csv")
df_2480

----------------------------------------df_2480 export to d:/tmp/e.csv----------------------------------------


,number,hira,level,multi_prop,prop1,sub_prop1,prop2,sub_prop2,prop3,sub_prop3,word_build,origin
word,,,,,,,,,,,,
亜,3,あ,6.0,,缀词(前),,,,,,名词-普通名词-一般,来自中文
アーチスト,9,あーちすと,3.0,,名词,,,,,,名词-普通名词-一般,来自西方
愛煙家,17,あいえんか,6.0,,名词,,,,,,名词-普通名词-一般 ＋ 缀词(后)-名词的-一般,来自中文
ＩＱ,20,あいきゅー,5.0,,名词,,,,,,名词-普通名词-一般,来自西方
アイスコーヒー,40,あいすこーひー,1.0,,名词,,,,,,名词-普通名词-一般 ＋ 名词-普通名词-一般,来自西方
...,...,...,...,...,...,...,...,...,...,...,...,...
割に,19305,わりに,4.0,,副词,,,,,,名词-普通名词-可变副词 ＋ 助词-格助词,日语固有词
割りばし,19306,わりばし,3.0,,名词,,,,,,名词-普通名词-一般,日语固有词
割引,20105,わりびき,5.0,〇,名词,,动词,动3,,,名词-普通名词-可变动3,日语固有词


In [52]:
to_csv_sig(df_jlpt_with_word_idx,"d:/tmp/f.csv")
p('df_jlpt_with_word_idx--ok')
df_jlpt_with_word_idx

df_jlpt_with_word_idx--ok


,number,hira,level,multi_prop,prop1,sub_prop1,prop2,sub_prop2,prop3,sub_prop3,word_build,origin
word,,,,,,,,,,,,
あ,4,あ,2.0,,名词,,,,,,名词-普通名词-一般,日语固有词
亜,3,あ,6.0,,缀词(前),,,,,,名词-普通名词-一般,来自中文
ああ,5,ああ,2.0,,感叹词,,,,,,感叹词-一般,日语固有词
ああ,6,ああ,3.0,,感叹词,,,,,,副词,日语固有词
アーケード,7,あーけーど,4.0,,名词,,,,,,名词-普通名词-一般,来自西方
...,...,...,...,...,...,...,...,...,...,...,...,...
ワンパターン,19336,わんぱたーん,5.0,,名词,,,,,,名词-普通名词-一般 ＋ 名词-普通名词-一般,来自西方
ワンピース,20121,わんぴーす,2.0,,名词,,,,,,名词-普通名词-一般,来自西方
ワンマン,19337,わんまん,5.0,,名词,,,,,,形状詞-一般,来自西方


In [53]:
df_jlpt_with_word_idx.index.name = "word_jlpt"
df_jlpt_word = (
    df_jlpt_with_word_idx.groupby("word_jlpt")
    .agg(
        {
             "prop1": lambda x: "|".join(x),
            # "sub_prop1": lambda x: "|".join(x.dropna().astype(str)),
            "sub_prop1": lambda x: "|".join(x[x != '']),
            "prop2": lambda x: "|".join(x[x != '']),
            "sub_prop2": lambda x: "|".join(x[x != '']),
            "prop3": lambda x: "|".join(x[x != '']),
            "sub_prop3": lambda x: "|".join(x[x != '']),
            "origin": lambda x: "|".join(x[x != '']),
            "level": lambda x: round(x.mean(), 1),
        }
    )  # 字符串拼接  # 求平均值
    .reset_index()
)
df_jlpt_word.rename(
    columns={
        "prop1": "prop1_jlpt",
        "sub_prop1": "sub_prop1_jlpt",
         "prop2": "prop2_jlpt",
        "sub_prop2": "sub_prop2_jlpt",
         "prop3": "prop3_jlpt",
        "sub_prop3": "sub_prop3_jlpt",
        "origin": "origin_jlpt",
    },
    inplace=True,
)
df_jlpt_word.set_index("word_jlpt", inplace=True)
to_csv_sig(df_jlpt_word, w_jlpt_word_csv, True)
p("df_jlpt_word--ok")

df_jlpt_word--ok


In [150]:
df_jlpt_with_multiidx

number  level multi_prop  prop1 sub_prop1 prop2 sub_prop2  \
word   hira                                                                
あ      あ            4    2.0                名词                             
亜      あ            3    6.0             缀词(前)                             
ああ     ああ           5    2.0               感叹词                             
       ああ           6    3.0               感叹词                             
アーケード  あーけーど        7    4.0                名词                             
...               ...    ...        ...    ...       ...   ...       ...   
ワンパターン わんぱたーん   19336    5.0                名词                             
ワンピース  わんぴーす    20121    2.0                名词                             
ワンマン   わんまん     19337    5.0                名词                             
腕力     わんりょく    19338    5.0                名词                             
わんわん   わんわん     19339    4.0                名词                             

              prop3 sub_prop3               word_build origin  cnt_multiindex  
word   hira                                                                    
あ      あ                                    名词-普通名词-一般  日语固有词               1  
亜      あ                                    名词-普通名词-一般   来自中文               1  
ああ     ああ                                       感叹词-一般  日语固有词               2  
       ああ                                           副词  日语固有词               2  
アーケード  あーけーど                                名词-普通名词-一般   来自西方               1  
...             ...       ...                      ...    ...             ...  
ワンパターン わんぱたーん                  名词-普通名词-一般 ＋ 名词-普通名词-一般   来自西方               1  
ワンピース  わんぴーす                                名词-普通名词-一般   来自西方               1  
ワンマン   わんまん                                     形状詞-一般   来自西方               1  
腕力     わんりょく                                名词-普通名词-一般   来自中文               1  
わんわん   わんわん                                         副词  日语固有词               1  

[17908 rows x 12 columns]

In [151]:
df_abc

,number,word,hira,level,multi_prop,prop1,sub_prop1,prop2,sub_prop2,prop3,sub_prop3,word_build,origin
0,4,あ,あ,2.0,,名词,,,,,,名词-普通名词-一般,日语固有词
1,3,亜,あ,6.0,,缀词(前),,,,,,名词-普通名词-一般,来自中文
2,5,ああ,ああ,2.0,,感叹词,,,,,,感叹词-一般,日语固有词
3,6,ああ,ああ,3.0,,感叹词,,,,,,副词,日语固有词
4,7,アーケード,あーけーど,4.0,,名词,,,,,,名词-普通名词-一般,来自西方
...,...,...,...,...,...,...,...,...,...,...,...,...,...
17903,19336,ワンパターン,わんぱたーん,5.0,,名词,,,,,,名词-普通名词-一般 ＋ 名词-普通名词-一般,来自西方
17904,20121,ワンピース,わんぴーす,2.0,,名词,,,,,,名词-普通名词-一般,来自西方
17905,19337,ワンマン,わんまん,5.0,,名词,,,,,,形状詞-一般,来自西方
17906,19338,腕力,わんりょく,5.0,,名词,,,,,,名词-普通名词-一般,来自中文


In [153]:
df_jlpt_multiidx_agg = (
    df_jlpt_with_multiidx.groupby(df_jlpt_with_multiidx.index)
    .agg(
        {
            "prop1": lambda x: "|".join(x),
            # "sub_prop1": lambda x: "|".join(x.dropna().astype(str)),
            "sub_prop1": lambda x: "|".join(x[x != '']),
            "prop2": lambda x: "|".join(x[x != '']),
            "sub_prop2": lambda x: "|".join(x[x != '']),
            "prop3": lambda x: "|".join(x[x != '']),
            "sub_prop3": lambda x: "|".join(x[x != '']),
            "origin": lambda x: "|".join(x[x != '']),
            "level": lambda x: round(x.mean(), 1),
            "cnt_multiindex": lambda x: max(x),
        }
    )  # 字符串拼接  # 求平均值
    .reset_index()
)


df_jlpt_multiidx_agg.rename(
    columns={
        "prop1": "prop1_jlpt",
        "sub_prop1": "sub_prop1_jlpt",
        "prop2": "prop2_jlpt",
        "sub_prop2": "sub_prop2_jlpt",
        "prop3": "prop3_jlpt",
        "sub_prop3": "sub_prop3_jlpt",
        "origin": "origin_jlpt",
        "cnt_multiindex": "cnt_multiindex_jlpt",
    },
    inplace=True,
)
# df_jlpt_multiidx_agg.set_index("word_jlpt", inplace=True)
to_csv_sig(df_jlpt_multiidx_agg, w_jlpt_multiidx_agg_csv)
p("df_jlpt_multiidx_agg--ok")
df_jlpt_multiidx_agg


df_jlpt_multiidx_agg--ok


,index,prop1_jlpt,sub_prop1_jlpt,prop2_jlpt,sub_prop2_jlpt,prop3_jlpt,sub_prop3_jlpt,origin_jlpt,level,cnt_multiindex_jlpt
0,"(あ, あ)",名词,,,,,,日语固有词,2.0,1
1,"(ああ, ああ)",感叹词|感叹词,,,,,,日语固有词|日语固有词,2.5,2
2,"(あいつ, あいつ)",代词,,,,,,日语固有词,4.0,1
3,"(あいづち, あいづち)",名词,,,,,,日语固有词,4.0,1
4,"(あいにく, あいにく)",副词,,,,,,日语固有词,5.0,1
...,...,...,...,...,...,...,...,...,...,...
17491,"(ＴＶ, てぃーぶい)",名词,,,,,,来自西方,3.0,1
17492,"(Ｖネック, ぶいねっく)",名词,,,,,,来自西方,5.0,1
17493,"(ＶＩＰ, ぶいあいぴー)",名词,,,,,,来自西方,5.0,1
17494,"(ＶＴＲ, ぶいてぃーあーる)",名词,,,,,,来自西方,4.0,1
